# DCOPF

In [1]:
import scipy
import sympy as smp
import numpy as np

import andes
import ams

In [2]:
andes.config_logger(stream_level=10)

In [3]:
sa = andes.load(andes.get_case('ieee14/ieee14_ace.xlsx'),
              setup=True,)

Working directory: "/Users/jinningwang/Documents/work/ams/dev/demo"
Found files: ['/Users/jinningwang/Documents/work/mambaforge/envs/ams/lib/python3.9/site-packages/andes/cases/ieee14/ieee14_ace.xlsx']
> Loaded config from file "/Users/jinningwang/.andes/andes.rc"
> Loaded generated Python code in "/Users/jinningwang/.andes/pycode".
Input format guessed as xlsx.
Parsing input file "/Users/jinningwang/Documents/work/mambaforge/envs/ams/lib/python3.9/site-packages/andes/cases/ieee14/ieee14_ace.xlsx"...
Input file parsed in 0.3175 seconds.
Setting internal address for Bus
Setting internal address for PQ
Setting internal address for PV
Setting internal address for Slack
Setting internal address for Shunt
Setting internal address for Line
Setting internal address for Area
System internal structure set up in 0.0254 seconds.


In [4]:
ams.config_logger(stream_level=10)

In [5]:
sp = ams.load(ams.get_case('matpower/case14.m'),
              setup=True,)

Working directory: "/Users/jinningwang/Documents/work/ams/dev/demo"
Found files: ['/Users/jinningwang/Documents/work/ams/ams/cases/matpower/case14.m']
Input format guessed as matpower.
Parsing input file "/Users/jinningwang/Documents/work/ams/ams/cases/matpower/case14.m"...
Detected raw file encoding: {'encoding': 'ascii', 'confidence': 1.0, 'language': ''}
CASE14    Power flow data for IEEE 14 bus test case.
Input file parsed in 0.0070 seconds.
System set up in 0.0010 seconds.


In [6]:
sp.DCOPF

Routine DCOPF at 0x14b57cfa0

In [7]:
sp.DCOPF.syms.generate_symbols()
sp.DCOPF.syms.generate_pretty_print()

- Generating symbols for DCOPF
- Generating pretty prints for DCOPF


In [8]:
sp.DCOPF.syms.inputs_dict

OrderedDict([('c2', c2),
             ('c1', c1),
             ('c0', c0),
             ('pmax', pmax),
             ('pmin', pmin),
             ('pd', pd),
             ('rate_a', rate_a),
             ('pg', p),
             ('sparselib', sparselib),
             ('linsolve', linsolve),
             ('sys_f', sys_f),
             ('sys_mva', sys_mva)])